### Download and Preprocess the Dataset

In [3]:
### IMPORTS
import pyreadstat
import pandas as pd
import pandas as pd
from scipy.stats import pearsonr, spearmanr, pointbiserialr
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats

Download the dataset

In [4]:
original_df, meta = pyreadstat.read_sas7bdat(r'data_donnee\hs.sas7bdat')

Filter and Preprocess the dataset

In [5]:
#'HWTDGHTM' = height column
#'HWTDGWTK' = weight column 
#'HWTDGBMI' = BMI column

df = original_df 
# The initial size of the dataframe
print(df.shape)

# Filter the dataframe
df['HWTDGHTM'] = pd.to_numeric(df['HWTDGHTM'], errors='coerce')
df['HWTDGBMI'] = pd.to_numeric(df['HWTDGBMI'], errors='coerce')
df['HWTDGWTK'] = pd.to_numeric(df['HWTDGWTK'], errors='coerce')

# Filter the dataframe
# Get rid of people who haven't answered
df = df[(df['HWTDGHTM'] < 2.5) & (df['HWTDGBMI'] < 60) & (df['HWTDGWTK'] < 150)]

# Filter so that you only have women in the dataframe
df = df[df['DHH_SEX'] == 2]

# Delete patients who are missing the chronic illness survey done 
df = df[df['DOCCC'] != 2]

# Delete patients who are younger than 18 years old
df = df[df['DHHGAGE'] > 2]

# Now, we will further filter the DataFrame to remove rows where 'CCC_045' is 7, 8, or 9.
# We only include rows that answered: "I have fibromyalgia" or "I don't have fibromyalgia"
df = df[df['CCC_045'].isin([1, 2])]

print(df.shape)
unique_values = df['CCC_045'].value_counts()
print(unique_values)

(109659, 1284)
(50028, 1284)
CCC_045
2.0    48133
1.0     1895
Name: count, dtype: int64


Our dataset has 109659 rows (patients) and 1284 columns (variables) before any filtering. After filtering men and subjects with unacceptable answers out, we are left with 53596. Out of 1895 of them have fibromyalgia.

Now, we will randomly select 1895 out of non-fibromyalgia subjects so that our statistical tests doesn't get skewed by the sheer size of the non-fibro sample.

In [6]:
# Filter the DataFrame to get rows where CCC_045 is 1.0 (yes fibromyalgia)
df_1 = df[df['CCC_045'] == 1.0]

# Filter the DataFrame to get rows where CCC_045 is 2.0 (no fibromyalgia)
df_2 = df[df['CCC_045'] == 2.0]

# Take a random sample of size 1897 from the 2.0 category
sample_df_2 = df_2.sample(n=1895, random_state=42)

# Combine the two DataFrames
filtered_df = pd.concat([df_1, sample_df_2])

# Replace values, now CCC_045 means that the patient doesn't have fibromyalgia
filtered_df['CCC_045'] = filtered_df['CCC_045'].replace(2, 0)

df = filtered_df

unique_values = filtered_df['CCC_045'].value_counts()
print(unique_values)

unique_values = filtered_df['DHHGAGE'].value_counts()
print(unique_values)

CCC_045
1.0    1895
0.0    1895
Name: count, dtype: int64
DHHGAGE
13.0    449
11.0    444
12.0    431
10.0    383
14.0    379
16.0    280
9.0     261
8.0     236
15.0    234
7.0     204
6.0     191
5.0     151
4.0     106
3.0      41
Name: count, dtype: int64


Now, we have 1895 patients from each category (people who have or people who doesn't have fibromyalgia)

Let's add an estimated skin area column to these subjects by using Du Bois formula: BSA(m^2)=0.007184 × ((Height(cm))^0.725) × ((Weight(kg))^0.425)

BSA = Body Surface Area

In [7]:
df['BSA'] = 0.007184 * (df['HWTDGHTM'] * 100) ** 0.725 * df['HWTDGWTK'] ** 0.425

### Check correlations of height, weight, bmi and skin area (bsa) with fibromyalgia

In [17]:
# Columns
height = df['HWTDGHTM']
weight = df['HWTDGWTK']
bmi = df['HWTDGBMI']
bsa = df['BSA']
fibromyalgia_status = df['CCC_045']

###########################    HEIGHT    #################################
# Pearson correlation
pearson_corr, pearson_p = pearsonr(height, fibromyalgia_status)
print(f'Height: Pearson correlation: {pearson_corr}, p-value: {pearson_p}')

# Spearman correlation
spearman_corr, spearman_p = spearmanr(height, fibromyalgia_status)
print(f'Spearman correlation: {spearman_corr}, p-value: {spearman_p}')

# Point-Biserial correlation (only if fibromyalgia_status is binary)
if df['CCC_045'].nunique() == 2:
    pointbiserial_corr, pointbiserial_p = pointbiserialr(fibromyalgia_status, height)
    print(f'Point-Biserial correlation: {pointbiserial_corr}, p-value: {pointbiserial_p} \n')
else:
    print('Point-Biserial correlation is not applicable as the fibromyalgia status variable is not binary.')

###########################    WEIGHT    #################################
# Pearson correlation
pearson_corr, pearson_p = pearsonr(weight, fibromyalgia_status)
print(f'WEIGHT: Pearson correlation: {pearson_corr}, p-value: {pearson_p}')

# Spearman correlation
spearman_corr, spearman_p = spearmanr(weight, fibromyalgia_status)
print(f'Spearman correlation: {spearman_corr}, p-value: {spearman_p}')

# Point-Biserial correlation (only if fibromyalgia_status is binary)
if df['CCC_045'].nunique() == 2:
    pointbiserial_corr, pointbiserial_p = pointbiserialr(fibromyalgia_status, weight)
    print(f'Point-Biserial correlation: {pointbiserial_corr}, p-value: {pointbiserial_p} \n')
else:
    print('Point-Biserial correlation is not applicable as the fibromyalgia status variable is not binary.')

###########################    BMI    #################################
# Pearson correlation
pearson_corr, pearson_p = pearsonr(bmi, fibromyalgia_status)
print(f'BMI: Pearson correlation: {pearson_corr}, p-value: {pearson_p}')

# Spearman correlation
spearman_corr, spearman_p = spearmanr(bmi, fibromyalgia_status)
print(f'Spearman correlation: {spearman_corr}, p-value: {spearman_p}')

# Point-Biserial correlation (only if fibromyalgia_status is binary)
if df['CCC_045'].nunique() == 2:
    pointbiserial_corr, pointbiserial_p = pointbiserialr(fibromyalgia_status, bmi)
    print(f'Point-Biserial correlation: {pointbiserial_corr}, p-value: {pointbiserial_p} \n')
else:
    print('Point-Biserial correlation is not applicable as the fibromyalgia status variable is not binary.')


###########################    SURFACE AREA    #################################
# Pearson correlation
pearson_corr, pearson_p = pearsonr(bsa, fibromyalgia_status)
print(f'BSA - ESTIMATED SKIN SURFACE AREA:\nPearson correlation: {pearson_corr}, p-value: {pearson_p}')

# Spearman correlation
spearman_corr, spearman_p = spearmanr(bsa, fibromyalgia_status)
print(f'Spearman correlation: {spearman_corr}, p-value: {spearman_p}')

# Point-Biserial correlation (only if fibromyalgia_status is binary)
if df['CCC_045'].nunique() == 2:
    pointbiserial_corr, pointbiserial_p = pointbiserialr(fibromyalgia_status, bsa)
    print(f'Point-Biserial correlation: {pointbiserial_corr}, p-value: {pointbiserial_p}')
else:
    print('Point-Biserial correlation is not applicable as the fibromyalgia status variable is not binary.')

Height: Pearson correlation: -0.048124182988062064, p-value: 0.00304253391776734
Spearman correlation: -0.04408040980270724, p-value: 0.006644702049321087
Point-Biserial correlation: -0.048124182988062064, p-value: 0.00304253391776734 

WEIGHT: Pearson correlation: 0.1659135101184542, p-value: 8.486000575701152e-25
Spearman correlation: 0.1671962750664145, p-value: 3.663633623736912e-25
Point-Biserial correlation: 0.1659135101184542, p-value: 8.486000575701152e-25 

BMI: Pearson correlation: 0.1875147513092818, p-value: 2.4740052245336616e-31
Spearman correlation: 0.1898266581088423, p-value: 4.4064604150301994e-32
Point-Biserial correlation: 0.1875147513092818, p-value: 2.4740052245336616e-31 

BSA - ESTIMATED SKIN SURFACE AREA:
Pearson correlation: 0.13278892713913593, p-value: 2.238592544270953e-16
Spearman correlation: 0.13419980291509626, p-value: 1.0716276526950227e-16
Point-Biserial correlation: 0.13278892713913593, p-value: 2.238592544270953e-16


##### All of the variables have weak (height being VERY weak) but statistically significant correlation to fibromyalgia (bmi - weight - bsa - height in order of strength high to low) 

Let's check the mean height/weight/bmi/bsa in fibro and non-fibro subjects

#### Partial Correlations controlled for other Independent Variables

In [9]:
import pandas as pd
from pingouin import partial_corr

# Load your data into a DataFrame (assuming the DataFrame is already loaded as df)
# df = pd.read_csv('your_data.csv')  # Modify this line as needed to load your data

# Assign columns to variables
height = df['HWTDGHTM']
weight = df['HWTDGWTK']
bmi = df['HWTDGBMI']
bsa = df['BSA']
fibromyalgia_status = df['CCC_045']

# Partial correlation between height and fibromyalgia status, controlling for weight, skin area, and BMI
partial_corr_height = partial_corr(data=df, x='HWTDGHTM', y='CCC_045', covar=['HWTDGWTK', 'BSA', 'HWTDGBMI'])
print("Partial correlation (height ~ fibromyalgia status | weight, skin area, BMI):")
print(partial_corr_height)

# Partial correlation between weight and fibromyalgia status, controlling for height, skin area, and BMI
partial_corr_weight = partial_corr(data=df, x='HWTDGWTK', y='CCC_045', covar=['HWTDGHTM', 'BSA', 'HWTDGBMI'])
print("Partial correlation (weight ~ fibromyalgia status | height, skin area, BMI):")
print(partial_corr_weight)

# Partial correlation between skin area (BSA) and fibromyalgia status, controlling for height, weight, and BMI
partial_corr_bsa = partial_corr(data=df, x='BSA', y='CCC_045', covar=['HWTDGHTM', 'HWTDGWTK', 'HWTDGBMI'])
print("Partial correlation (skin area ~ fibromyalgia status | height, weight, BMI):")
print(partial_corr_bsa)

# Partial correlation between BMI and fibromyalgia status, controlling for height, weight, and skin area
partial_corr_bmi = partial_corr(data=df, x='HWTDGBMI', y='CCC_045', covar=['HWTDGHTM', 'HWTDGWTK', 'BSA'])
print("Partial correlation (BMI ~ fibromyalgia status | height, weight, skin area):")
print(partial_corr_bmi)


Partial correlation (height ~ fibromyalgia status | weight, skin area, BMI):
            n         r          CI95%     p-val
pearson  3790 -0.033423  [-0.07, -0.0]  0.039717
Partial correlation (weight ~ fibromyalgia status | height, skin area, BMI):
            n         r          CI95%     p-val
pearson  3790 -0.006459  [-0.04, 0.03]  0.691119
Partial correlation (skin area ~ fibromyalgia status | height, weight, BMI):
            n         r          CI95%     p-val
pearson  3790  0.023955  [-0.01, 0.06]  0.140512
Partial correlation (BMI ~ fibromyalgia status | height, weight, skin area):
            n         r          CI95%     p-val
pearson  3790 -0.001249  [-0.03, 0.03]  0.938751


None of the independent variables (height, weight, skin area, BMI) show a strong or statistically significant direct relationship with fibromyalgia status after controlling for the other variables. The only significant result is the partial correlation between height and fibromyalgia status, but the effect size is very small and might not be practically significant.

#### Compare the means

In [20]:
# Define the columns
height = df['HWTDGHTM']
weight = df['HWTDGWTK']
bmi = df['HWTDGBMI']
bsa = df['BSA']
age = df['DHHGAGE']
fibromyalgia_status = df['CCC_045']

# Separate the data into two groups
fibro_group = df[df['CCC_045'] == 1]
non_fibro_group = df[df['CCC_045'] == 0]

# Calculate means
mean_height_fibro = fibro_group['HWTDGHTM'].mean()
mean_weight_fibro = fibro_group['HWTDGWTK'].mean()
mean_BMI_fibro = fibro_group['HWTDGBMI'].mean()
mean_BSA_fibro = fibro_group['BSA'].mean()
mean_age_fibro = fibro_group['DHHGAGE'].mean()

mean_height_non_fibro = non_fibro_group['HWTDGHTM'].mean()
mean_weight_non_fibro = non_fibro_group['HWTDGWTK'].mean()
mean_BMI_non_fibro = non_fibro_group['HWTDGBMI'].mean()
mean_BSA_non_fibro = non_fibro_group['BSA'].mean()
mean_age_non_fibro = non_fibro_group['DHHGAGE'].mean()


# Perform t-tests
t_height, p_height = stats.ttest_ind(fibro_group['HWTDGHTM'], non_fibro_group['HWTDGHTM'], equal_var=False)
t_weight, p_weight = stats.ttest_ind(fibro_group['HWTDGWTK'], non_fibro_group['HWTDGWTK'], equal_var=False)
t_BMI, p_BMI = stats.ttest_ind(fibro_group['HWTDGBMI'], non_fibro_group['HWTDGBMI'], equal_var=False)
t_BSA, p_BSA = stats.ttest_ind(fibro_group['BSA'], non_fibro_group['BSA'], equal_var=False)
t_age, p_age = stats.ttest_ind(fibro_group['DHHGAGE'], non_fibro_group['DHHGAGE'], equal_var=False)

# Print the results
print("Mean height of people with fibromyalgia:", mean_height_fibro)
print("Mean weight of people with fibromyalgia:", mean_weight_fibro)
print("Mean BMI of people with fibromyalgia:", mean_BMI_fibro)
print("Mean BSA of people with fibromyalgia:", mean_BSA_fibro,)
print("Mean age of people with fibromyalgia:", mean_age_fibro, "\n")

print("Mean height of people without fibromyalgia:", mean_height_non_fibro)
print("Mean weight of people without fibromyalgia:", mean_weight_non_fibro)
print("Mean BMI of people without fibromyalgia:", mean_BMI_non_fibro)
print("Mean BSA of people without fibromyalgia:", mean_BSA_non_fibro)
print("Mean age of people without fibromyalgia:", mean_age_non_fibro, "\n")


# Print the results
print(f"Height: t-statistic = {t_height:.4f}, p-value = {p_height:.4f}")
print(f"Weight: t-statistic = {t_weight:.4f}, p-value = {p_weight:.4f}")
print(f"BMI: t-statistic = {t_BMI:.4f}, p-value = {p_BMI:.4f}")
print(f"BSA: t-statistic = {t_BSA:.4f}, p-value = {p_BSA:.4f}")
print(f"Age: t-statistic = {t_age:.4f}, p-value = {p_age:.4f}")


Mean height of people with fibromyalgia: 1.6163018469656991
Mean weight of people with fibromyalgia: 73.93981530343008
Mean BMI of people with fibromyalgia: 28.315540897097623
Mean BSA of people with fibromyalgia: 1.7764724587004308
Mean age of people with fibromyalgia: 11.624274406332454 

Mean height of people without fibromyalgia: 1.6228559366754618
Mean weight of people without fibromyalgia: 68.7397889182058
Mean BMI of people without fibromyalgia: 26.117356200527706
Mean BSA of people without fibromyalgia: 1.7283371064681714
Mean age of people without fibromyalgia: 10.081266490765172 

Height: t-statistic = -2.9653, p-value = 0.0030
Weight: t-statistic = 10.3550, p-value = 0.0000
BMI: t-statistic = 11.7493, p-value = 0.0000
BSA: t-statistic = 8.2457, p-value = 0.0000
Age: t-statistic = 14.8236, p-value = 0.0000


People who have fibromyalgia are, in average, older, shorter, heavier, has a higher BMI and skin area than those who don't have fibromyalgia. These differences are statistically significant as shown in the t-test.

Age is divided into categories in our dataset: category 10 -> 50-54 , category 11 -> 55-59

It is established that BMI and age positively correlate with fibromyalgia, in the following section we will take our new variables like heihgt, weight, skin area and see if they have an effect on the condition independent from BMI and Age

#### Height and BMI as independent predictors of fibromylagia
We will use logistic regression to see if height and bmi are independently predicting fibromyalgia, suggesting if each has its own effect on the disease

In [33]:
# Only height and bmi model
X = df[['HWTDGHTM', 'HWTDGBMI']]
y = df['CCC_045']

# LOGISTIC REGRESSION
# Adding a constant for the intercept
X = sm.add_constant(X)

model = sm.Logit(y, X).fit()
print(model.summary())

# Calculating VIF for each predictor
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)


Optimization terminated successfully.
         Current function value: 0.665031
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                CCC_045   No. Observations:                 3794
Model:                          Logit   Df Residuals:                     3791
Method:                           MLE   Df Model:                            2
Date:                Mon, 15 Jul 2024   Pseudo R-squ.:                 0.04056
Time:                        02:02:25   Log-Likelihood:                -2523.1
converged:                       True   LL-Null:                       -2629.8
Covariance Type:            nonrobust   LLR p-value:                 4.697e-47
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2758      0.821     -0.336      0.737      -1.885       1.333
HWTDGHTM      -1.1710      0.

(?) Coefficient of height shows that it is a better predictor than bmi, while both being statistically significant. 

The odds ratio represents the change in odds of the dependent variable (the likelihood of having fibromyalgia) for a one-unit increase in the predictor variable, holding all other variables constant.

Height (HWTDGHTM) (per meters):
Coefficient per meter: -1.1710
Odds Ratio: 
e^−1.1710 ≈ 0.310

Coefficient per cm = −1.1710/100 = −0.01171
e^-0.01171 ≈ 0.988

BMI (HWTDGBMI):
Coefficient: 0.0809
Odds Ratio: 
e^0.0809 ≈ 1.084

Interpretation:

For height, the odds ratio of 0.310 indicates that for each unit (per meters) increase in height (in meters), the odds of having fibromyalgia decrease by approximately 69% (since 1 - 0.310 = 0.69).
For each centimeter increase in height, the odds of having fibromyalgia decrease by about 1.2% (since 1 - 0.988 = 0.012 or 1.2%).

For BMI, the odds ratio of 1.084 suggests that for each unit increase in BMI, the odds of having fibromyalgia increase by about 8.4%.

The VIF (Variance Inflation Factor) values for both height and BMI are around 1.014, indicating that multicollinearity is not a concern in this model.

(?) It is suprising how height has weaker correlation but is a better predictor of fibromyalgia

In [24]:
# height, bmi, age
X = df[['HWTDGHTM', 'HWTDGBMI', 'DHHGAGE']]
y = df['CCC_045']

# LOGISTIC REGRESSION
# Adding a constant for the intercept
X = sm.add_constant(X)

model = sm.Logit(y, X).fit()
print(model.summary())

# Calculating VIF for each predictor
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

Optimization terminated successfully.
         Current function value: 0.648490
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                CCC_045   No. Observations:                 3790
Model:                          Logit   Df Residuals:                     3786
Method:                           MLE   Df Model:                            3
Date:                Mon, 15 Jul 2024   Pseudo R-squ.:                 0.06443
Time:                        15:48:12   Log-Likelihood:                -2457.8
converged:                       True   LL-Null:                       -2627.0
Covariance Type:            nonrobust   LLR p-value:                 4.612e-73
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -5.0899      0.914     -5.566      0.000      -6.882      -3.298
HWTDGHTM       1.0147      0.

##### It seems that hight looses it statistical significany (even though by a very close margin) when we include age. This might be due to the fact that people tends to lose some height as they grow older after a certain age because of the spine compression and posture problems.

### Weight and BMI and Age as independent predictors of fibromyalgia

In [39]:
X = df[['HWTDGWTK', 'HWTDGBMI']]
y = df['CCC_045']

# LOGISTIC REGRESSION
# Adding a constant for the intercept
X = sm.add_constant(X)

model = sm.Logit(y, X).fit()
print(model.summary())

# Calculating VIF for each predictor
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)


Optimization terminated successfully.
         Current function value: 0.665019
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                CCC_045   No. Observations:                 3794
Model:                          Logit   Df Residuals:                     3791
Method:                           MLE   Df Model:                            2
Date:                Mon, 15 Jul 2024   Pseudo R-squ.:                 0.04058
Time:                        02:28:20   Log-Likelihood:                -2523.1
converged:                       True   LL-Null:                       -2629.8
Covariance Type:            nonrobust   LLR p-value:                 4.494e-47
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.1714      0.164    -13.217      0.000      -2.493      -1.849
HWTDGWTK      -0.0135      0.

Wight (HWTDGWTK) (per kilograms):
Coefficient per meter: -0.0135
Odds Ratio: 
e^−0.0135 ≈ 0.987


BMI (HWTDGBMI):
Coefficient:  0.1162
Odds Ratio: 
e^ 0.1162 ≈ 1.123

Interpretation:

For weight, the odds ratio of 0.987 indicates that for each unit increase in weight (in kilograms), the odds of having fibromyalgia decrease by approximately 1.3% (since 1 - 0.987 = 0.013 or 1.3%).
For BMI, the odds ratio of 1.123 suggests that for each unit increase in BMI, the odds of having fibromyalgia increase by about 12.3%.
The VIF (Variance Inflation Factor) values for both weight and BMI are around 6.970, indicating some level of multicollinearity, but it is generally considered acceptable as it is below 10.

In [25]:
X = df[['HWTDGWTK', 'HWTDGBMI', 'DHHGAGE']]
y = df['CCC_045']

# LOGISTIC REGRESSION
# Adding a constant for the intercept
X = sm.add_constant(X)

model = sm.Logit(y, X).fit()
print(model.summary())

# Calculating VIF for each predictor
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

Optimization terminated successfully.
         Current function value: 0.648461
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                CCC_045   No. Observations:                 3790
Model:                          Logit   Df Residuals:                     3786
Method:                           MLE   Df Model:                            3
Date:                Mon, 15 Jul 2024   Pseudo R-squ.:                 0.06447
Time:                        15:55:53   Log-Likelihood:                -2457.7
converged:                       True   LL-Null:                       -2627.0
Covariance Type:            nonrobust   LLR p-value:                 4.122e-73
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.4490      0.211    -16.319      0.000      -3.863      -3.035
HWTDGWTK       0.0117      0.

Weight (HWTDGWTK):
Coefficient: 0.0117
Odds Ratio: 
e^0.0117 ≈ 1.012

BMI (HWTDGBMI):
Coefficient: 0.0356
Odds Ratio:
e^0.0356 ≈ 1.036
 
Age (DHHGAGE):
Coefficient: 0.1515
Odds Ratio: 
e^0.1515 ≈ 1.163


Interpretation:

For weight, the odds ratio of 1.012 indicates that for each unit increase in weight (in kilograms), the odds of having fibromyalgia increase by approximately 1.2%.
For BMI, the odds ratio of 1.036 suggests that for each unit increase in BMI, the odds of having fibromyalgia increase by about 3.6%.
For age, the odds ratio of 1.163 indicates that for each unit increase in age (in years), the odds of having fibromyalgia increase by approximately 16.3%.
The VIF (Variance Inflation Factor) values indicate that there is some multicollinearity between weight and BMI (with VIFs around 7), but it is generally considered acceptable as it is below 10. Age shows a low VIF, indicating low multicollinearity with other variables.

In our case a unit increase in age is as follows: 
Age between 18 and 19 - 03 
Age between 20 and 24 - 04 
Age between 25 and 29 - 05
Age between 30 and 34 - 06
Age between 35 and 39 - 07 
Age between 40 and 44 - 08 
Age between 45 and 49 - 09 
Age between 50 and 54 - 10 
Age between 55 and 59 - 11 
Age between 60 and 64 - 12 
Age between 65 and 69 - 13 
Age between 70 and 74 - 14 
Age between 75 and 79 - 15 
Age 80 and older      - 16

##### The fact that predictive power of weight stays stable but BMI lessens when we include age might be due to the fact that people grow shorter after a certain age which affects BMI.

### Weight and Skin Area and Age as independent predictors of fibromyalgia

In [26]:
X = df[['HWTDGWTK', 'BSA']]
y = df['CCC_045']

# LOGISTIC REGRESSION
# Adding a constant for the intercept
X = sm.add_constant(X)

model = sm.Logit(y, X).fit()
print(model.summary())

# Calculating VIF for each predictor
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

Optimization terminated successfully.
         Current function value: 0.675323
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                CCC_045   No. Observations:                 3790
Model:                          Logit   Df Residuals:                     3787
Method:                           MLE   Df Model:                            2
Date:                Mon, 15 Jul 2024   Pseudo R-squ.:                 0.02571
Time:                        18:05:54   Log-Likelihood:                -2559.5
converged:                       True   LL-Null:                       -2627.0
Covariance Type:            nonrobust   LLR p-value:                 4.597e-30
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.6185      0.611      2.651      0.008       0.422       2.815
HWTDGWTK       0.0588      0.

Since there is significant multicollinearity, our coefficient estimates are not reliable.

In [30]:
X = df[['HWTDGWTK', 'BSA', "DHHGAGE"]]
y = df['CCC_045']

# LOGISTIC REGRESSION
# Adding a constant for the intercept
X = sm.add_constant(X)

model = sm.Logit(y, X).fit()
print(model.summary())

# Calculating VIF for each predictor
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

Optimization terminated successfully.
         Current function value: 0.648472
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                CCC_045   No. Observations:                 3790
Model:                          Logit   Df Residuals:                     3786
Method:                           MLE   Df Model:                            3
Date:                Mon, 15 Jul 2024   Pseudo R-squ.:                 0.06445
Time:                        18:10:49   Log-Likelihood:                -2457.7
converged:                       True   LL-Null:                       -2627.0
Covariance Type:            nonrobust   LLR p-value:                 4.303e-73
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.9527      0.677     -2.883      0.004      -3.280      -0.625
HWTDGWTK       0.0401      0.

In [29]:
X = df[['HWTDGWTK', 'BSA']]
y = df['CCC_045']

# LOGISTIC REGRESSION
# Adding a constant for the intercept
X = sm.add_constant(X)

# Adding the interaction term to the model
X['weight_BSA_interaction'] = X['HWTDGWTK'] * X['BSA']

interaction_model = sm.Logit(y, X).fit()
print(interaction_model.summary())

model = sm.Logit(y, X).fit()
print(model.summary())

# Calculating VIF for each predictor
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

Optimization terminated successfully.
         Current function value: 0.674861
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                CCC_045   No. Observations:                 3790
Model:                          Logit   Df Residuals:                     3786
Method:                           MLE   Df Model:                            3
Date:                Mon, 15 Jul 2024   Pseudo R-squ.:                 0.02638
Time:                        18:09:09   Log-Likelihood:                -2557.7
converged:                       True   LL-Null:                       -2627.0
Covariance Type:            nonrobust   LLR p-value:                 7.541e-30
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     -0.4567      1.264     -0.361      0.718      -2.934      

Also didn't work

##### Didn't achieve any significant result from weight x skin area x age

### Skin Area and BMI and Age as independent predictors of fibromyalgia

In [31]:
X = df[['BSA', 'HWTDGBMI']]
y = df['CCC_045']

# LOGISTIC REGRESSION
# Adding a constant for the intercept
X = sm.add_constant(X)

model = sm.Logit(y, X).fit()
print(model.summary())

# Calculating VIF for each predictor
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

Optimization terminated successfully.
         Current function value: 0.674980
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                CCC_045   No. Observations:                 3790
Model:                          Logit   Df Residuals:                     3787
Method:                           MLE   Df Model:                            2
Date:                Mon, 15 Jul 2024   Pseudo R-squ.:                 0.02621
Time:                        18:11:20   Log-Likelihood:                -2558.2
converged:                       True   LL-Null:                       -2627.0
Covariance Type:            nonrobust   LLR p-value:                 1.250e-30
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.4373      0.354     -4.065      0.000      -2.130      -0.744
BSA           -0.3466      0.

Effect of BSA is not statistically significant

This might be a counter-argument for the small fiber neuropathy since individual with larger skin areas have more small fibers. - > Not really: The hypothesis suggests that fibromyalgia patients may have a reduced density of small nerve fibers or dysfunctional small fibers, leading to pain and other sensory symptoms. This reduction in density or dysfunction can occur regardless of the total skin area.



In [34]:
X = df[['BSA', 'HWTDGBMI','DHHGAGE']]
y = df['CCC_045']

# LOGISTIC REGRESSION
# Adding a constant for the intercept
X = sm.add_constant(X)

model = sm.Logit(y, X).fit()
print(model.summary())

# Calculating VIF for each predictor
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

Optimization terminated successfully.
         Current function value: 0.648492
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                CCC_045   No. Observations:                 3790
Model:                          Logit   Df Residuals:                     3786
Method:                           MLE   Df Model:                            3
Date:                Mon, 15 Jul 2024   Pseudo R-squ.:                 0.06442
Time:                        18:13:23   Log-Likelihood:                -2457.8
converged:                       True   LL-Null:                       -2627.0
Covariance Type:            nonrobust   LLR p-value:                 4.640e-73
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.0484      0.413     -9.792      0.000      -4.859      -3.238
BSA            0.5894      0.

Effect of BSA is not statistically significant

In [11]:
# Define the independent variables
X = df[['HWTDGHTM', 'HWTDGWTK', 'HWTDGBMI', 'BSA', 'DHHGAGE']]
X = sm.add_constant(X)  # Adds a constant term to the model

# Define the dependent variable
y = df['CCC_045']

# Fit the logistic regression model
logit_model = sm.Logit(y, X)
result = logit_model.fit()

# Print the summary of the logistic regression model
print(result.summary())

# Calculating VIF for each predictor
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

Optimization terminated successfully.
         Current function value: 0.648444
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                CCC_045   No. Observations:                 3790
Model:                          Logit   Df Residuals:                     3784
Method:                           MLE   Df Model:                            5
Date:                Sun, 21 Jul 2024   Pseudo R-squ.:                 0.06449
Time:                        00:47:49   Log-Likelihood:                -2457.6
converged:                       True   LL-Null:                       -2627.0
Covariance Type:            nonrobust   LLR p-value:                 4.391e-71
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.5522      4.022     -0.883      0.377     -11.435       4.331
HWTDGHTM       0.7506      3.